In [15]:
# Import necessary modules
import os
import time
import math
import random
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Define time functions
def fConvertMilis(seconds):
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%dh %02dm %02ds" % (hour, minutes, seconds)
# Define the scraping function
def fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot):
    fStartTime = time.time()
    print("---------{}.) {}--------".format(serviceNumber, service))
    # Set the Driver up
    # Instantiate an Options object
    option = webdriver.ChromeOptions()
    #Remove navigator.webdriver flag
    option.add_argument('--disable-blink-features=AutomationControlled')
    # Change the resolution of the browser
    option.add_argument("window-size=1920,1080")
    # Adjusting the user agent
    option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
    #Open Browser
    driver = webdriver.Chrome(executable_path='chromedriver.exe', options=option)
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    # Close the popup
    if pPopup:
        bClosePopUp = driver.find_element_by_xpath(pPopup).click()
    # Open the "Select Language" dropdown menu
    if ddSelectLanguage: 
        ddSource = driver.find_element_by_xpath(ddSelectLanguage).click()
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    # Click the English button
    if ddbEnglish:
        button = driver.find_element_by_xpath(ddbEnglish).click()
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    # Find the text field
    textFieldSource = driver.find_element_by_xpath(tSourceInput)
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    if bRobot:
        # Type the dummy text into the text field to trigger the bot detection
        textFieldSource.send_keys('我喜欢吃冰淇淋.')
        # Sleep to allow for manual bot-detectiong passing
        time.sleep(30)
        # Sleep to avoid errors
        time.sleep(random.uniform(2.0, 4.0))
        # Find the text field and clear it
        textFieldSource = driver.find_element_by_xpath(tSourceInput).clear()
        # Open the "Select Language" dropdown menu
        buttonSource = driver.find_element_by_xpath(ddSelectLanguage).click()
        # Sleep to avoid errors
        time.sleep(random.uniform(2.0, 4.0))
        # Click the English button
        button = driver.find_element_by_xpath(ddbEnglish).click()
        # Find the text field once more
        textFieldSource = driver.find_element_by_xpath(tSourceInput)
    # Type the text into the text field
    batch = 1
    sentencesCleanBatches = {}
    lSentencesClean = []
    nLimit = 4300
    nDeepLimit = 1000
    nQQLimit = 1800
    # Get number of rows
    sTotalRows = source.shape[0]
    for index, row in source.iterrows():
        # Input the row's text
        textFieldSource = driver.find_element_by_xpath(tSourceInput).send_keys(source['origin'][index])
        # Sleep to avoid errors
        #time.sleep(random.uniform(0.1, 0.5))
        # Create a new line
        textFieldSource = driver.find_element_by_xpath(tSourceInput).send_keys(Keys.RETURN)
        # Get the number of chars to the current row
        oNumChars = source['origin'][0:index].str.len().sum()   
        # Check to see if the site is DeepL     
        if 'deepl' in url and oNumChars >= nDeepLimit:
            # Pause to let the translations load
            time.sleep(5)
            # Increment the limit for the next batch
            nDeepLimit += 1000
        if 'qq' in url and oNumChars >= nQQLimit:
            # Increment the limit for the next batch
            nQQLimit += 1800
            # Pause to avoid errors
            time.sleep(10)
            # Store the translated text as a string
            if "text" in oExtractMethod:
                translatedText = driver.find_element_by_xpath(tTranslatedText).text
            else:
                translatedText = driver.find_element_by_xpath(tTranslatedText).get_attribute('innerHTML').strip()
            # Sleep to avoid errors
            time.sleep(random.uniform(2,3))
            # Clean the output text and print
            sentencesCleanBatches[batch] = translatedText.split("\n")
            # Print the batch number and its length        
            print(f"Batch {batch}: {len(sentencesCleanBatches[batch])}")
            # Increment the batch number
            batch += 1
            # Find the text field once more and clear it
            if 'google' in url:
                # Open the specfied URL
                driver.get(url)
                # Sleep to avoid errors
                time.sleep(random.uniform(5,7))
            else:
                driver.refresh()
        # Check to see if the number of chars is too much
        if oNumChars > nLimit:
            # Increment the limit for the next batch
            nLimit += 4300
            # Pause to avoid errors
            time.sleep(15)
            # Store the translated text as a string
            if "text" in oExtractMethod:
                translatedText = driver.find_element_by_xpath(tTranslatedText).text
            else:
                translatedText = driver.find_element_by_xpath(tTranslatedText).get_attribute('innerHTML').strip()
            # Sleep to avoid errors
            time.sleep(random.uniform(2,3))
            # Clean the output text and print
            sentencesCleanBatches[batch] = translatedText.split("\n")
            # Print the batch number and its length        
            print(f"Batch {batch}: {len(sentencesCleanBatches[batch])}")
            # Increment the batch number
            batch += 1
            # Find the text field once more and clear it
            if 'google' in url:
                # Open the specfied URL
                driver.get(url)
                # Sleep to avoid errors
                time.sleep(random.uniform(5,7))
            else:
                driver.refresh()
    # Pause to avoid errors
    time.sleep(15)
    # Store the translated text as a string
    if "text" in oExtractMethod:
        translatedText = driver.find_element_by_xpath(tTranslatedText).text
    else:
        translatedText = driver.find_element_by_xpath(tTranslatedText).get_attribute('innerHTML').strip()
    # Sleep to avoid errors
    time.sleep(random.uniform(2,3))
    # Clean the output for the last batch
    sentencesCleanBatches[batch] = translatedText.split("\n")
    # Print the batch number and its length        
    print(f"Batch {batch}: {len(sentencesCleanBatches[batch])}")
    # Concatenate all the batches into once list
    for i in range(1, batch + 1):
        lSentencesClean = lSentencesClean + sentencesCleanBatches[i]
    # Make our returned object equal to the above list
    dSentencesClean[serviceNumber] = lSentencesClean
    # Sleep to avoid errors
    time.sleep(2)
    # Remove period strings in the list
    noPeriods = dSentencesClean[serviceNumber]
    while("." in noPeriods): 
        noPeriods.remove(".")
    dNoPeriods[serviceNumber] = noPeriods
    # Strip extra things in the list
    stripped = noPeriods
    stripped = [item.strip() for item in stripped]
    dStripped[serviceNumber] = stripped
    # Remove empty strings in the list
    noEmpty = stripped
    while("" in noEmpty): 
        noEmpty.remove("")
    dNoEmpty[serviceNumber] = noEmpty
    # Remove elipses
    while("..." in noEmpty):
        noEmpty.remove("...")
    noElipses = noEmpty
    # Define the dictionary DF
    dSentencesCleanFinal[serviceNumber] = noElipses
    fEndTime = time.time()
    # Print to console for the report
    print("Rows: {}".format(sTotalRows))
    print("Duration: {}".format(fConvertMilis(fEndTime - fStartTime)))
    print("Length: {}".format(len(dSentencesCleanFinal[serviceNumber])))
    # Close the driver
    driver.close()
    # Return said DF
    return dSentencesCleanFinal[serviceNumber], sTotalRows, dNoPeriods[serviceNumber], dStripped[serviceNumber], dNoEmpty[serviceNumber], dSentencesClean[serviceNumber]

## The Program

### User Inputs

In [2]:
# USER INPUTS
# Define the source file
sourceFile = 'tishi 989 - 1592.xlsx'
# Define the start row
vStartRow = 989
# Define the end row
vEndRow = 1592
# Define the project name
project = 'tishi ({}-{})'.format(vStartRow, vEndRow)
# Define the language
language = '中文'

# Create a DataFrame of the sentences to be translated
source = pd.read_excel(sourceFile, header = 0)
# Name the columns so we can manipulate them
source.columns = ['ID', 'origin', 'blah1', 'blah2']
# Drop the columns we don't need
source.drop(columns = ['blah1', 'blah2'], inplace = True)
# Remove \n
source['origin'] = source['origin'].replace(r'\\n', ' ', regex = True)
# Index the necessary rows
source = source[vStartRow-1:vEndRow].reset_index()
# Get number of rows
sTotalRows = source.shape[0]

### Run it!

In [3]:
# Define start time
oStartTime = time.time()
# Define service number to start with
serviceNumber = 0
# Create a dictionary to store the lists
dSentencesClean = {}
dNoPeriods = {}
dStripped = {}
dNoEmpty = {}
dSentencesCleanFinal = {}
# Create a list to store the service names
lServices = ['source']
# Define the source column
dSentencesCleanFinal[0] = source['origin']

# Baidu
serviceNumber += 1 
service = 'Baidu'
url = 'https://fanyi.baidu.com/#zh/en/'
ddSelectLanguage = False
ddbEnglish = False
tSourceInput = '//*[@id="baidu_translate_input"]'
tTranslatedText = '//*[@id="main-outer"]/div/div/div[1]/div[2]/div[1]/div[2]/div/div/div[1]'
pPopup = '/html/body/div[1]/div[7]/div/div/a[2]'
oExtractMethod = "text"
bRobot = False
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# DeepL
serviceNumber += 1 
service = 'DeepL'
url = 'https://www.deepl.com/translator'
ddSelectLanguage = False
ddbEnglish = False
tSourceInput = '//*[@id="dl_translator"]/div[5]/div[2]/div[1]/div[2]/div[1]/textarea'
tTranslatedText = '//*[@id="target-dummydiv"]'
pPopup = False
oExtractMethod = "get_attribute('innerHTML').strip()"
bRobot = False
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# Google
serviceNumber += 1 
service = 'Google'
url = 'https://translate.google.com/'
ddSelectLanguage = False
ddbEnglish = False
tSourceInput = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[1]/span/span/div/textarea'
tTranslatedText = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[2]/div[5]/div/div[1]'
pPopup = False
oExtractMethod = "text"
bRobot = False
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# SoGou
serviceNumber += 1 
service = 'SoGou'
url = 'https://translate.sogou.com/text'
ddSelectLanguage = '//*[@id="J-langselect"]/div/span[3]'
ddbEnglish = '/html/body/div/div/div[2]/div[1]/div/div[2]/div[4]/div[1]/span[2]'
tSourceInput = '//*[@id="trans-input"]'
tTranslatedText = '//*[@id="trans-result"]'
pPopup = False
oExtractMethod = "get_attribute('innerHTML').strip()"
bRobot = True
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# # QQ
# serviceNumber += 1
# service = 'QQ'
# url = 'https://fanyi.qq.com/'
# ddSelectLanguage = '//*[@id="language-button-group-target"]/div[1]'
# ddbEnglish = '//*[@id="language-button-group-target"]/div[2]/ul/li[2]/span'
# tSourceInput = '/html/body/div[2]/div[2]/div[2]/div[1]/div[1]/textarea'
# tTranslatedText = '/html/body/div[2]/div[2]/div[2]/div[2]/div[2]'
# pPopup = False
# oExtractMethod = "text"
# bRobot = False
# fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
# lServices.append(service)

# CREATE A DATAFRAME
# Define the DF
df = pd.DataFrame(columns = lServices)
# Define the index variable
x = 0
# Populate the ID column
df['ID'] = source['index']
# Populate the other columns
for i in lServices:
    df[i] = dSentencesCleanFinal[x] 
    x += 1
# Drop nan rows
df.dropna(how = 'all', inplace = True, axis = 0)
# Export DataFrame as an XLSX to Google Drive\Work\Translation folder
df.to_excel('{}\{}.xlsx'.format(language, project), index=False)
os.startfile('{}\{}.xlsx'.format(language, project))
# Define end time
oEndTime = time.time()
print("~~~~~REPORT~~~~~")
print("Scraped {} rows of {} in {} for {} services.".format(sTotalRows, language, fConvertMilis(oEndTime - oStartTime), serviceNumber))

---------1.) Baidu--------
Batch 1: 395
Batch 2: 209
Rows: 604
Duration: 0h 02m 25s
Length: 604
---------2.) DeepL--------
Batch 1: 395
Batch 2: 209
Rows: 604
Duration: 0h 02m 51s
Length: 604
---------3.) Google--------
Batch 1: 396
Batch 2: 210
Rows: 604
Duration: 0h 04m 19s
Length: 604
---------4.) SoGou--------
Batch 1: 395
Batch 2: 209
Rows: 604
Duration: 0h 02m 52s
Length: 604
Scraped 604 rows of 中文 in 0h 12m 45s for 4 services.


## Testing

In [6]:
dSentencesCleanFinal[0] = source['origin']
x = 0
for i in lServices:
    print("{} ({}): {}".format(i, x, len(dSentencesCleanFinal[x])))
    x += 1

source (0): 604
Baidu (1): 604
DeepL (2): 604
Google (3): 604
SoGou (4): 604


In [16]:
# SoGou
serviceNumber += 1 
service = 'SoGou'
url = 'https://translate.sogou.com/text'
ddSelectLanguage = '//*[@id="J-langselect"]/div/span[3]'
ddbEnglish = '/html/body/div/div/div[2]/div[1]/div/div[2]/div[4]/div[1]/span[2]'
tSourceInput = '//*[@id="trans-input"]'
tTranslatedText = '//*[@id="trans-result"]'
pPopup = False
oExtractMethod = "get_attribute('innerHTML').strip()"
bRobot = True
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

---------6.) SoGou--------
Batch 1: 395
Batch 2: 209
Rows: 604
Duration: 0h 03m 16s
Length: 604


In [21]:
# Define the DF
df = pd.DataFrame(columns = lServices)
# Define the index variable
x = 0
# Populate the ID column
df['ID'] = source['index']
# Populate the other columns
for i in lServices:
    df[i] = dSentencesCleanFinal[x] 
    x += 1
# Drop nan rows
df.dropna(how = 'all', inplace = True, axis = 0)
# Export DataFrame as an XLSX to Google Drive\Work\Translation folder
df.to_excel('{}\{}.xlsx'.format(language, project), index=False)
os.startfile('{}\{}.xlsx'.format(language, project))
# Define end time
oEndTime = time.time()
print("~~~~~REPORT~~~~~")
print("Scraped {} rows of {} in {} for {} services.".format(sTotalRows, language, fConvertMilis(oEndTime - oStartTime), serviceNumber))

~~~~~REPORT~~~~~
Scraped 604 rows of 中文 in 0h 29m 38s for 6 services.
